
# 1️⃣ What Are Tools in LLM?

A **tool** is simply a function that the LLM can call when it needs external help.

LLMs:

* ❌ Cannot access real-time data
* ❌ Cannot call APIs directly
* ❌ Cannot access your database
* ❌ Cannot run code

So we give them **TOOLS**.

Think of it like:

> 🧠 LLM = Brain
> 🛠 Tools = Hands

The LLM decides:

> “I need to call this function.”

Then your backend:

* Executes the function
* Sends the result back to the LLM
* LLM continues the conversation

---

# 2️⃣ What Is Happening in YOUR Code?

You are doing:

```python
response = openai.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools
)
```

This means:

> “Model, here are some tools you are allowed to use.”

If the model decides it needs one, it returns:

```python
finish_reason == "tool_calls"
```

That means:

> “I want to call a function.”

Then you manually:

* Read the tool call
* Execute your Python function
* Send the result back
* Ask the model again

That loop here:

```python
while response.choices[0].finish_reason=="tool_calls":
```

This is exactly how an **AI Agent loop works**.

---

# 3️⃣ How To Create a Tool Properly

A tool has 2 parts:

---

## ✅ Part 1 — Tool Definition (Schema)

This tells the LLM:

* Function name
* Description
* Parameters
* Parameter types

Example:

```python
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_ticket_price",
            "description": "Get the ticket price for a destination city",
            "parameters": {
                "type": "object",
                "properties": {
                    "destination_city": {
                        "type": "string",
                        "description": "The name of the city"
                    }
                },
                "required": ["destination_city"]
            }
        }
    }
]
```

This is VERY important.

This is how the model understands:

* When to use the tool
* What arguments to pass

---

## ✅ Part 2 — Actual Python Function

```python
def get_ticket_price(destination_city):
    tickets = {
        "delhi": 5000,
        "mumbai": 4500,
        "chennai": 6000
    }
    price = tickets.get(destination_city.lower(), "City not found")
    return f"The ticket price to {destination_city} is {price} rupees."
```

The LLM never sees this code directly.
You execute it when the LLM requests it.

---

# 4️⃣ How Tool Calling Actually Works (Step-by-Step Flow)

Let’s say user says:

> “How much is a ticket to Delhi?”

### Step 1:

LLM sees tool definition.

It thinks:

> I should call get_ticket_price.

It responds:

```json
{
  "tool_calls": [
    {
      "function": {
        "name": "get_ticket_price",
        "arguments": {
          "destination_city": "Delhi"
        }
      }
    }
  ]
}
```

---

### Step 2:

Your backend executes:

```python
get_ticket_price("Delhi")
```

---

### Step 3:

You send result back:

```python
{
  "role": "tool",
  "content": "The ticket price to Delhi is 5000 rupees.",
  "tool_call_id": ...
}
```

---

### Step 4:

LLM continues and generates final response.

---

# 5️⃣ Why Tools Are Powerful

Tools allow LLM to:

* 🔍 Search internet
* 💾 Query database
* 📧 Send email
* 🖼 Generate image
* 📅 Add calendar event
* 📊 Run SQL query
* 📂 Read files
* 🤖 Control robots (like your EMG project 😉)

This is how real AI agents are built.

---

# 6️⃣ Important Concepts You Must Understand

### 🔹 1. Tool Selection Is Done By Model

You don't force it (unless you specify tool_choice).

### 🔹 2. Tools Need Clear Descriptions

Bad description → Model won't call correctly.

### 🔹 3. Always Validate Arguments

Never trust model blindly:

```python
arguments = json.loads(tool_call.function.arguments)
```

Always validate.

---

# 7️⃣ Best Practices (Very Important)

### ✅ 1. Make descriptions clear

Bad:

```python
"description": "Gets price"
```

Good:

```python
"description": "Returns the ticket price in rupees for a given Indian city."
```

---

### ✅ 2. Use strict parameter schema

Include:

* type
* required
* description

---

### ✅ 3. Always handle errors safely

Example:

```python
if not city:
    return "Invalid city provided."
```

---

### ✅ 4. Keep tool logic separate from LLM logic

Your architecture is good:

* chat() handles LLM
* handle_tool_calls_and_return_cities() handles execution

That’s clean design 👌

---

# 8️⃣ How This Connects to LangChain

LangChain just automates this loop:

Instead of:

```python
while finish_reason == "tool_calls":
```

LangChain does:

* Tool parsing
* Execution
* Memory
* Re-asking model

Under the hood, it’s the SAME concept.



# 1️⃣ Multiple Tools

### What It Means

Instead of giving the LLM one function, you give it many.

Example:

```python
tools = [
    get_ticket_price_tool,
    get_weather_tool,
    book_flight_tool,
    generate_image_tool
]
```

Now the model decides:

* Which tool to use
* Whether to use any tool at all

---

### What Happens Internally

The model:

1. Reads the user query
2. Reads tool descriptions
3. Matches intent → best tool
4. Generates structured JSON call

It is not "if-else logic".

It is **semantic matching**.

Example:

User:

> “How much is a ticket to Mumbai?”

Model thinks:

* This is pricing information
* Tool with description mentioning "ticket price" is best
* Calls `get_ticket_price`

---

### Why This Is Powerful

You’re moving from:

> Chatbot

to

> AI Decision Engine

---

### Production Advice

When you have multiple tools:

* Descriptions must be very clear
* Avoid overlapping responsibilities
* Keep tools atomic (one responsibility per tool)

---

# 2️⃣ Tool Chaining

This is where things get interesting.

Tool chaining =
One tool’s output becomes input for another tool.

---

### Example

User:

> “Book the cheapest ticket to the warmest city tomorrow.”

Steps needed:

1. Call weather tool → find warmest city
2. Call ticket tool → get prices
3. Choose cheapest
4. Call booking tool

That is multi-step reasoning.

---

### How It Happens

There are 2 approaches:

---

## 🔹 Approach A: Let Model Decide Step-by-Step

Loop:

* Model calls tool
* You send result
* Model decides next tool
* Continue until no more tool calls

Your while loop already supports this.

---

## 🔹 Approach B: You Orchestrate Steps

You control order in backend:

```python
city = get_warmest_city()
price = get_ticket_price(city)
book_ticket(city)
```

This is more controlled.

---

### Key Understanding

Tool chaining = reasoning across tools.

This is the foundation of:

* Autonomous agents
* Multi-step AI workflows
* Task decomposition systems

---

# 3️⃣ Tool Selection Logic

By default:
Model chooses tool automatically.

But you can control it.

---

### Option A — Automatic (Default)

```python
openai.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools
)
```

Model decides.

---

### Option B — Force Specific Tool

```python
tool_choice={"type": "function", "function": {"name": "get_ticket_price"}}
```

Now model MUST call that tool.

---

### Option C — Disable Tools

```python
tool_choice="none"
```

Forces pure text response.

---

### Production Strategy

Use:

* Automatic for general agents
* Forced for structured workflows

Example:
If you're building a SQL agent,
You ALWAYS want model to call query tool.

---

# 4️⃣ Tool Forcing (Very Important)

Tool forcing is used when:

* You need structured data
* You don't want hallucinations
* You require deterministic behavior

---

Example:

Instead of asking:

> “Extract name and age.”

You define:

```python
{
  "name": "extract_user_info",
  "parameters": {
    "name": {"type": "string"},
    "age": {"type": "integer"}
  }
}
```

Then force tool call.

Now model MUST return structured JSON.

This eliminates parsing errors.

---

This is how you build:

* Resume parsers
* Invoice extraction systems
* JSON-based pipelines

---

# 5️⃣ Parallel Tool Calls

Modern LLMs can call multiple tools at once.

Example:

User:

> “Tell me the weather in Delhi and Mumbai.”

Model may generate:

```json
tool_calls: [
  { "name": "get_weather", "arguments": {"city": "Delhi"} },
  { "name": "get_weather", "arguments": {"city": "Mumbai"} }
]
```

Now you execute both.

Your loop:

```python
for tool_call in message.tool_calls:
```

Already supports this.

---

### Why This Matters

Parallel calls:

* Improve performance
* Reduce round-trips
* Enable batch operations

In production, this saves API costs and latency.

---

# 6️⃣ Structured Output Tools

This is extremely powerful.

Instead of free text, you force the model to output structured JSON via a tool.

---

Example: Job extractor

Tool schema:

```python
{
  "name": "extract_job_info",
  "parameters": {
    "title": {"type": "string"},
    "company": {"type": "string"},
    "location": {"type": "string"},
    "deadline": {"type": "string"}
  }
}
```

Now model returns guaranteed structured output.

This is far more reliable than:

> “Return JSON please”

Because natural text can break format.

---

### Why This Is Critical For You

For your:

* Internship tracking platform
* Email PDF metadata extractor
* AI annotation pipeline

Structured tool outputs = clean database writes.

---

# 7️⃣ Building Autonomous Agents

Now we reach the highest level.

An autonomous agent:

* Plans tasks
* Uses tools
* Evaluates results
* Decides next action
* Stops when goal achieved

---

### Agent Loop Architecture

1. User gives goal
2. Model creates plan
3. Model selects tool
4. Execute tool
5. Send result back
6. Model reflects
7. Repeat until done

This is called:

ReAct Pattern
(Reason + Act)

---

### Minimal Agent Pseudocode

```python
while True:
    response = call_llm(messages, tools)

    if no_tool_call:
        break

    execute_tool()
    append_tool_result()
```

That’s it.

That loop is the heart of all agents.

LangChain, AutoGPT, CrewAI — same idea.

---

# 🔬 Deeper Concept: Why Tools Work So Well

Because LLMs are very good at:

* Intent detection
* Function argument extraction
* Planning
* Decomposing tasks

But very bad at:

* Real-time data
* Deterministic computation
* Memory beyond context
* Secure execution

Tools combine:

* LLM reasoning
* Deterministic backend logic

That hybrid model is modern AI architecture.

---

# 🧠 Final Mental Model

Think of it like this:

LLM = Brain
Tools = APIs
Agent Loop = Nervous System
Backend = Body

You are building the nervous system.
